In [1]:
# build ofa resnet50
from ofa.model_zoo import ofa_net
from torchvision import transforms, datasets
import os
import torch
import torch.nn as nn
from torchvision import transforms, datasets
import numpy as np
import time
import random
import math
import copy
#from matplotlib import pyplot as plt
from ofa.nas.search_algorithm import EvolutionFinder


ofa_network = ofa_net('ofa_resnet50', pretrained=True)


In [2]:
# accuracy predictor
import torch
from ofa.nas.accuracy_predictor import AccuracyPredictor, ResNetArchEncoder
from ofa.utils import download_url

image_size_list = [128, 144, 160, 176, 192, 224, 240, 256]
arch_encoder = ResNetArchEncoder(
	image_size_list=image_size_list, depth_list=ofa_network.depth_list, expand_list=ofa_network.expand_ratio_list,
    width_mult_list=ofa_network.width_mult_list, base_depth_list=ofa_network.BASE_DEPTH_LIST
)

acc_predictor_checkpoint_path = download_url(
    'https://hanlab.mit.edu/files/OnceForAll/tutorial/ofa_resnet50_acc_predictor.pth',
    model_dir='~/.ofa/',
)
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
acc_predictor = AccuracyPredictor(arch_encoder, 400, 3,
                                  checkpoint_path=acc_predictor_checkpoint_path, device=device)

print('The accuracy predictor is ready!')
print(acc_predictor)

Loaded checkpoint from /home/panyj/.ofa/ofa_resnet50_acc_predictor.pth
The accuracy predictor is ready!
AccuracyPredictor(
  (layers): Sequential(
    (0): Sequential(
      (0): Linear(in_features=82, out_features=400, bias=True)
      (1): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Linear(in_features=400, out_features=400, bias=True)
      (1): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Linear(in_features=400, out_features=400, bias=True)
      (1): ReLU(inplace=True)
    )
    (3): Linear(in_features=400, out_features=1, bias=False)
  )
)


In [3]:
# build efficiency predictor
from ofa.nas.efficiency_predictor import ResNet50FLOPsModel

efficiency_predictor = ResNet50FLOPsModel(ofa_network)

from ofa.nas.memory_predictor import ResNet50WorkingMemModel 
memory_predictor_baseline = ResNet50WorkingMemModel(ofa_network, 0)
memory_predictor_ideal = ResNet50WorkingMemModel(ofa_network, 1)
memory_predictor_self = ResNet50WorkingMemModel(ofa_network, 2)


In [4]:
# path to the ImageNet dataset
imagenet_data_path = '/data2/jiecaoyu/imagenet/imgs/'
print('The ImageNet dataset files are ready.')



The ImageNet dataset files are ready.


In [5]:
# The following function build the data transforms for test
def build_val_transform(size):
    return transforms.Compose([
        transforms.Resize(int(math.ceil(size / 0.875))),
        transforms.CenterCrop(size),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ),
    ])

data_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(
        root=os.path.join(imagenet_data_path, 'val'),
        transform=build_val_transform(224)
    ),
    batch_size=250,  # test batch size
    shuffle=True,
    num_workers=12,  # number of workers for the data loader
    pin_memory=True,
    drop_last=False,
)
print('The ImageNet dataloader is ready.')

The ImageNet dataloader is ready.


In [ ]:
""" Hyper-parameters for the evolutionary search process
    You can modify these hyper-parameters to see how they influence the final ImageNet accuracy of the search sub-net.
"""
FLOPs_constraint = 2000  # MFLOPs
workingmem_constraint = 400 # KB
P = 100  # The size of population in each generation
N = 200  # How many generations of population to be searched
r = 0.25  # The ratio of networks that are used as parents for next generation
params = {
    #'constraint_type': target_hardware, # Let's do FLOPs-constrained search
    #'efficiency_constraint': FLOPs_constraint,
    'mutate_prob': 0.1, # The probability of mutation in evolutionary search
    'mutation_ratio': 0.5, # The ratio of networks that are generated through mutation in generation n >= 2.
    'efficiency_predictor': efficiency_predictor, # To use a predefined efficiency predictor.
    'accuracy_predictor': acc_predictor, # To use a predefined accuracy_predictor predictor.
    'memory_predictor': memory_predictor_baseline, # To use a predefined working memory predictor
    'population_size': P,
    'max_time_budget': N,
    'parent_ratio': r,
}

# build the evolution finder
finder = EvolutionFinder(**params)

# start searching
result_lis = []
st = time.time()
best_valids, best_info = finder.run_evolution_search(FLOPs_constraint, workingmem_constraint, verbose=True)
result_lis.append(best_info)
ed = time.time()
print('Found best architecture with FLOPS <= %.2f M and working mem <= %.2f in %.2f seconds! '
      'It achieves %.2f%s predicted accuracy ' %
      (FLOPs_constraint, workingmem_constraint, ed-st, best_info[0] * 100, '%'))

# visualize the architecture of the searched sub-net
_, net_config, FLOPS, workingmem = best_info
ofa_network.set_active_subnet(w=net_config['w'], d=net_config['d'], e=net_config['e'])
print('Architecture of the searched sub-net:')
print(ofa_network.module_str)
print('FLOPS',FLOPS,'workingmem', workingmem)

In [ ]:
FLOPs_constraint = 2000  # MFLOPs
workingmem_constraint = 400 # KB
P = 100  # The size of population in each generation
N = 200  # How many generations of population to be searched
r = 0.25  # The ratio of networks that are used as parents for next generation
params = {
    #'constraint_type': target_hardware, # Let's do FLOPs-constrained search
    #'efficiency_constraint': FLOPs_constraint,
    'mutate_prob': 0.1, # The probability of mutation in evolutionary search
    'mutation_ratio': 0.5, # The ratio of networks that are generated through mutation in generation n >= 2.
    'efficiency_predictor': efficiency_predictor, # To use a predefined efficiency predictor.
    'accuracy_predictor': acc_predictor, # To use a predefined accuracy_predictor predictor.
    'memory_predictor': memory_predictor_ideal, # To use a predefined working memory predictor
    'population_size': P,
    'max_time_budget': N,
    'parent_ratio': r,
}

# build the evolution finder
finder = EvolutionFinder(**params)

# start searching
result_lis = []
st = time.time()
best_valids, best_info = finder.run_evolution_search(FLOPs_constraint, workingmem_constraint, verbose=True)
result_lis.append(best_info)
ed = time.time()
print('Found best architecture with FLOPS <= %.2f M and working mem <= %.2f in %.2f seconds! '
      'It achieves %.2f%s predicted accuracy ' %
      (FLOPs_constraint, workingmem_constraint, ed-st, best_info[0] * 100, '%'))

# visualize the architecture of the searched sub-net
_, net_config, FLOPS, workingmem = best_info
ofa_network.set_active_subnet(w=net_config['w'], d=net_config['d'], e=net_config['e'])
print('Architecture of the searched sub-net:')
print(ofa_network.module_str)
print('FLOPS',FLOPS,'workingmem', workingmem)

In [6]:
FLOPs_constraint = 2000  # MFLOPs
workingmem_constraint = 350 # KB
P = 100  # The size of population in each generation
N = 200  # How many generations of population to be searched
r = 0.25  # The ratio of networks that are used as parents for next generation
params = {
    #'constraint_type': target_hardware, # Let's do FLOPs-constrained search
    #'efficiency_constraint': FLOPs_constraint,
    'mutate_prob': 0.1, # The probability of mutation in evolutionary search
    'mutation_ratio': 0.5, # The ratio of networks that are generated through mutation in generation n >= 2.
    'efficiency_predictor': efficiency_predictor, # To use a predefined efficiency predictor.
    'accuracy_predictor': acc_predictor, # To use a predefined accuracy_predictor predictor.
    'memory_predictor': memory_predictor_self, # To use a predefined working memory predictor
    'population_size': P,
    'max_time_budget': N,
    'parent_ratio': r,
}

# build the evolution finder
finder = EvolutionFinder(**params)

# start searching
result_lis = []
st = time.time()
best_valids, best_info = finder.run_evolution_search(FLOPs_constraint, workingmem_constraint, verbose=True)
result_lis.append(best_info)
ed = time.time()
print('Found best architecture with FLOPS <= %.2f M and working mem <= %.2f in %.2f seconds! '
      'It achieves %.2f%s predicted accuracy ' %
      (FLOPs_constraint, workingmem_constraint, ed-st, best_info[0] * 100, '%'))

# visualize the architecture of the searched sub-net
_, net_config, FLOPS, workingmem = best_info
ofa_network.set_active_subnet(w=net_config['w'], d=net_config['d'], e=net_config['e'])
print('Architecture of the searched sub-net:')
print(ofa_network.module_str)
print('FLOPS',FLOPS,'workingmem', workingmem)

Generate random population...


KeyboardInterrupt: 